# APO Work Items - Wrangle

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Assess

### Workitems

In [ ]:
workitems = pd.read_csv('workitems.csv')
workitems.head()

In [ ]:
workitems.info()

In [ ]:
workitems.created_date = pd.to_datetime(workitems.created_date)

workitems.info()

In [ ]:
workitems.estimate.hist()

In [ ]:
workitems[['created_date', 'estimate']].describe(include = 'all')

In [ ]:
workitems[workitems.id.duplicated()]

In [ ]:
workitems[workitems.estimate == 0]

In [ ]:
workitems[workitems.estimate > 50]

In [ ]:
workitems.last_state.value_counts()

In [ ]:
workitems.workitem_type.value_counts()

In [ ]:
workitems.iteration_path.value_counts().sort_index()

In [ ]:
workitems[workitems.iteration_path.str.contains('Queue')]

In [ ]:
workitems.assigned_to.value_counts()

In [ ]:
workitems[workitems.assigned_to.isna()]

### Revisions

In [ ]:
revisions = pd.read_csv('revisions.csv')

In [ ]:
revisions.info()

In [ ]:
revisions.describe()

In [ ]:
revisions.head()

In [ ]:
revisions.id.nunique()

In [ ]:
revisions[revisions.id == 1733729]

In [ ]:
revisions[revisions.id == 887472]

In [ ]:
revisions[revisions.state == 'Active'][['id', 'state']].nunique()

#### Notes

**Quality**

* zero estimates
* missing assigned_to
* simplify assigned_to
* non standard workitem types
* remove rows with last_state not closed or resolved
* simplify assigned_to in revisions
* workitems in queue are included
* workitems assigned to not part of the team
* workitems not passed to active state
* revisions has multiple duplicate states

**Tidiness**

* extract dates from iteration path
* remove created date in workitems and replace it with what in revisions
* remove assigned to in workitems and replace it with what in revisions
* add important states on workitems


## Clean

In [ ]:
workitems_clean = workitems.copy()
revisions_clean = revisions.copy()

### Quality

#### Zero Estimates

*Set Zero Estimates to np.NaN*

Zero estimates means no effort, and it's impossible to have them. Set them to `np.NaN`

**Code**

In [ ]:
workitems_clean.loc[workitems_clean.estimate == 0, 'estimate'] = np.NaN

**Test**

In [ ]:
workitems_clean.estimate.value_counts()

In [ ]:
workitems_clean.info()

In [ ]:
workitems_clean.estimate.hist(bins=50)

#### Non standard workitem types
*Standardize naming for work item types*

Names are different from the first part of work items and the second part, to standardize name we need to change:
* Change Request -> Story

**Code**

In [ ]:
workitems_clean.loc[workitems_clean.workitem_type == 'Change Request', 'workitem_type']  = 'Story'

**Test**

In [ ]:
workitems_clean.workitem_type.value_counts()

#### Workitems rows with last_state not closed or resolved

*Remove last_state rows not closed or resolved* 

**Code**

In [ ]:
workitems_clean = workitems_clean[workitems_clean.last_state.isin(['Closed', 'Resolved'])]

**Test**

In [ ]:
workitems_clean.last_state.value_counts()

#### Workitems in queue are included

*There workitems that are in Queue or Backlog in iteration path. Those means that they are not worked on therefore we need to delete them*

**Code**

In [ ]:
workitems_clean[workitems_clean.iteration_path.str.contains('Queue') | workitems_clean.iteration_path.str.contains('Backlog') ]

In [ ]:
workitems_clean = workitems_clean[~(workitems_clean.iteration_path.str.contains('Queue') | workitems_clean.iteration_path.str.contains('Backlog'))]

**Test**

In [ ]:
len(workitems_clean[workitems_clean.iteration_path.str.contains('Queue') | workitems_clean.iteration_path.str.contains('Backlog')])

In [ ]:
workitems_clean.info()

#### `assigned_to` in revisions can be simplified for consistency
*Extract the user id as it is simple and consistent to reduce error during analysis*

**Code**

In [ ]:
pat = r'.*<\bCORP\b\\(?P<assigned_to>\w+)>'
revisions_clean.assigned_to = revisions_clean.assigned_to.str.extract(pat = pat)

**Test**

In [ ]:
revisions_clean.assigned_to.value_counts()

#### Revisions has 2 confusing states, `kanban` and `states`
*Standardize state by merging them, then, map to the following*
* if same kanban and state, just simplify them
* if state > kanban, follow state
* if kanban < state, follow kanban

*However, during analysis, we might be better just use `state` and discard `kanban`*

**Code**

In [ ]:
(revisions_clean[~revisions_clean['kanban'].isna()].kanban + ' - ' + revisions_clean[~revisions_clean['kanban'].isna()].state).unique()

In [ ]:
revisions_clean.drop(['kanban'], axis = 1, inplace=True)

**Test**

In [ ]:
revisions_clean.head()

#### Revisions has multiple state value
*Remove duplicates following these rules:*
* `Proposed`, `Active` - Get only the first
* `Resolved`, `Closed`, `Moved` - Get only the last

**Code**

In [ ]:
revisions_clean.changed_date = pd.to_datetime(revisions_clean.changed_date)

In [ ]:
start_state = revisions_clean[revisions_clean.state.isin(['Active', 'Proposed'])].groupby(['id', 'state'])['changed_date'].min().reset_index().sort_values(by = ['changed_date'])
print(len(start_state))
start_state = pd.merge(revisions_clean, start_state, on = ['id', 'state', 'changed_date'])
start_state.head()

In [ ]:
end_state = revisions_clean[revisions_clean.state.isin(['Resolved', 'Closed', 'Moved'])].groupby(['id', 'state'])['changed_date'].max().reset_index().sort_values(by = ['changed_date'])
print(len(end_state))
end_state = pd.merge(revisions_clean, end_state, on = ['id', 'state', 'changed_date'])
end_state.head(10)

In [ ]:
revisions_clean = pd.concat([start_state, end_state], ignore_index=True, sort=False)
revisions_clean = revisions_clean.sort_values(['id','changed_date']).reset_index(drop=True)

**Test**

In [ ]:
revisions_clean

In [ ]:
revisions_clean[revisions_clean.id == 797499]

In [ ]:
revisions[revisions.id == 797499]

### Tidiness

#### Remove columns retrievable from revisions

*remove the columns, `assigned_to`, `created_date`, `last_state`*

**Code**

In [ ]:
workitems_clean = workitems_clean[['id', 'workitem_type', 'estimate', 'iteration_path', 'title']]

**Test**

In [ ]:
workitems_clean.info()

In [ ]:
workitems_clean.head()

#### Dates in iteration path

*Extract dates in iteration path and move them to `iter_year`, `iter_num`, `iter_start` and `iter_end`*

**Code**

In [ ]:
workitems_clean.iteration_path.value_counts()

In [ ]:
#workitems.iteration_path.str.extract(r'NCA\\APO\\(?P<iter_year>\d{4})\\Iteration\s(?P<iter>\d+)\s\((?P<iter_startday>\d+)(?P<iter_startmon>\w{3})-(?P<iter_endday>\d+)(?P<iter_endmon>\w{3})\)')
workitems_clean[['iteration_path']] = workitems_clean.iteration_path.str.replace(r'NCA\\*', '', regex = True)
workitems_clean[['iteration_path']] = workitems_clean.iteration_path.str.replace(r'APO\\*', '', regex = True)
workitems_clean.head()

In [ ]:
pat = r'^(?P<year>\d+)'
workitems_clean[['year']] = workitems_clean.iteration_path.str.extract(pat)
rep = r'^\d+\\*'
workitems_clean[['iteration_path']] = workitems_clean.iteration_path.str.replace(rep, '')
workitems_clean[['iteration_path', 'year']]

In [ ]:
rep = r'^Iteration\s'
workitems_clean[['iteration_path']] = workitems_clean.iteration_path.str.replace(rep, '')

In [ ]:
rep = r'^\d\.\d+\\Sprint\s'
workitems_clean[['iteration_path']] = workitems_clean.iteration_path.str.replace(rep, '')

In [ ]:
workitems_clean.loc[workitems_clean.iteration_path == '1.12', 'iteration_path'] = ''

In [ ]:
pat = r'^(?P<sprint>\d+)(?:\s*\((?P<start_day>\d+)\s*(?P<start_month>[a-zA-Z]+)(?:\s*-*\s*(?P<end_day>\d+)-*(?P<end_month>[a-zA-Z]+))*)*'
workitems_clean[['sprint', 'start_day', 'start_month', 'end_day', 'end_month']] = workitems_clean.iteration_path.str.extract(pat)

In [ ]:
workitems_clean.drop(['iteration_path'], axis = 1, inplace=True)

**Test**

In [ ]:
workitems_clean.head()

In [ ]:
workitems_clean.info()

#### Title not analysis friendly

*Tokenize the title and remove articles from the title and store it as comma delimited list*

**Code**

In [ ]:
workitems_clean.title = workitems_clean.title.str.replace('\[.*\]', '')

In [ ]:
def cat(words):
    forbidden = ['the', 'can', 'for', 'not', 'out', 'and', 'for', 'all', 'must', 'are', 'does', 'more', 'has', 'only', 'how', 'when', 'should', 'from']
    result = [word for word in words if len(word) > 2 and word not in forbidden]
    result.sort()
    result = np.unique(result)
    return ",".join(result)
    
workitems_clean['words'] = workitems_clean.title.str.lower().str.split('[\W_]+').apply(cat)
workitems_clean.drop(['title'], axis = 1, inplace = True)

**Test**

In [ ]:
workitems_clean.head()

In [ ]:
workitems_clean.words

#### Melt state on revisions
*Melt the state into columns, then remove columns with no active state, remove also the moved column*

**Code**

In [ ]:
revisions_clean.head()

In [ ]:
revisions_pivot = revisions_clean.pivot(index = 'id', columns = 'state', values = 'changed_date').reset_index()
revisions_pivot.head()

In [ ]:
revisions_pivot[~revisions_pivot.Moved.isna()]

In [ ]:
revisions_pivot = revisions_pivot[revisions_pivot.Moved.isna()]
revisions_pivot.head()

In [ ]:
revisions_pivot.drop('Moved', axis = 1, inplace = True)
revisions_pivot.head()

In [ ]:
revisions_pivot = revisions_pivot[~revisions_pivot.Active.isna()]
revisions_pivot.head()

In [ ]:
revisions_pivot = revisions_pivot[~(revisions_pivot.Closed.isna() & revisions_pivot.Resolved.isna())]
revisions_clean.head()

In [ ]:
revisions_pivot = revisions_pivot.rename(str.lower, axis = 'columns')
revisions_pivot.head()

In [ ]:
assigned = revisions_clean[revisions_clean.state == 'Active'][['id', 'assigned_to']]
revisions_clean = pd.merge(assigned, revisions_pivot, on = ['id'], how = 'inner')

In [ ]:
revisions_clean = revisions_clean[~revisions_clean.assigned_to.isna()]

**Test**

In [ ]:
revisions_clean.info()

In [ ]:
len(revisions_clean[revisions_clean.closed.isna() & revisions_clean.resolved.isna()])

In [ ]:
len(revisions_clean[revisions_clean.active.isna()])

#### Merge Revisions with WorkItems
*Merge fields from revisions and workitems, use inner join*

**Code**

In [ ]:
workitems_clean = pd.merge(workitems_clean, revisions_clean, on='id', how = 'inner')

**Test**

In [ ]:
workitems_clean.info()

In [ ]:
workitems_clean.head()

## Save

In [ ]:
workitems.to_csv('workitems_master.csv', index=False)